## Min-Priority Queue ADT

- A queue where each element has a *priority*
- Operations:
    - `enqueue(elem, priority)`: insert elem with a given priority
    - `find_min`: return the element with the smallest priority
    - `extract_min`: remove the element with the smallest priority and return it

- Example usage: assign lower numbers to patients in the emergency room who need to be seen sooner

In [1]:
class PriorityQueue:
    def __init__(self):
        self.queue = []

    def insert(self, data, priority):
        self.queue.append((data, priority))

    def extract_min(self):
        min_idx = 0
        for i in range(len(self.queue)):
            if self.queue[i][1] < self.queue[min_idx][1]:
                min_idx = i

        return self.queue.pop(min_idx)[0]
    
if __name__ == '__main__':
    pq = PriorityQueue()
    pq.insert("Mike" , 3)
    pq.insert("Bob", 2)
    pq.insert("Alice", 1)
    pq.insert("Eve", 5)

    print(pq.extract_min()) # Alice
    print(pq.extract_min()) # Bob

Alice
Bob


## Shortest Paths

- Given a weighted connected graph `G = (V, E)` and a pair of vertices `V`<sub>`s`</sub>, `V`<sub>`d`</sub> `∈ V`, what is the shortest path between `V`<sub>`s`</sub> and `V`<sub>`d`</sub>?
    - Path with the smallest sum of edge weights

<img src="assets/intro.png"></img>

### Approach
- Shortest path (SP) from A to H = SP from A to E + SP from E to H
- SP from A to H = `min`<sub>`i`</sub>`(SP from A to V`<sub>`i`</sub> + `SP from V`<sub>`i`</sub> `to H)`

<img src="assets/approach.png"></img>

## Dijkstra's Algorithm

```python
Dijkstra(G = (V, E), source):
    S = {source} # S is the set of explored nodes
    d = {source} = 0 # d(v) is the shortest path from source to v


    while S != V:
        Choose v ∈ V s.t. d(u) + |(u, v)| is minimized, (u ∈ S)
        Add v to S, set d(v) = d(u) + |(u, v)|
```

<img src="assets/djikstra.png">

### Djikstra Complexity
- Depends on the implementation details
- Simplest implementation
    - To add one vertex to S, search through all possible additional vertices
    - `O(|V|`<sup>`2`</sup>`)`

- Fancier implementation
    - Add potential v’s to a priority queue as S grows
    - `O((|E|)log|V|)`

### Recovering the path:

```python
Dijkstra(G = (V, E), source):
    S = {source} # S is the set of explored nodes
    d = {source} = 0 # d(v) is the shortest path from source to v


    while S != V:
        Choose v ∈ V s.t. d(u) + |(u, v)| is minimized, (u ∈ S)
        Add v to S, set d(v) = d(u) + |(u, v)|
        prev(v) = u #Add this line here to recover the path -- record u as the node
```

### Idea:
- Maintain the distances from `S`to neighbours to `S`
- When we add a vertex `v` to `S`, only need to compute the distances of neighbours `v` to `S`
- Maintain a priority queue with the closest neighbour of `S` at the top

### Inefficient Implementation:

In [ ]:
'''
Example based on graph above

Visited nodes: S = {A, C, B, E, D, G, F}

d(A) = 0
d(C) = 1
d(B) = 2
d(E) = 3
d(D) = 6
d(G) = 8
d(F) = 10
d(H) = 15
'''
import numpy as np

class Node:
    def __init__(self, value):
        self.value = value
        self.connections = []
        self.distance_from_start = np.inf

class Connection:
    def __init__(self, node: Node, weight: float):
        self.node = node
        self.weight = weight

def Dijkstra(start, end):
    start.distance_from_start = 0
    visited = set([start])
    current = start 
    
    while current != end:
        cur_dist = np.inf
        cur_v = None
        for node in visited:
            for connection in node.connections:
                if connection.node in visited:
                    continue
                if cur_dist > node.distance_from_start + connection.weight:
                    cur_dist = node.distance_from_start + connection.weight
                    cur_v = connection.node

        current = cur_v
        current.distance_from_start = cur_dist
        visited.add(current)
    
    return current.distance_from_start

### Priority Queue Implementation of Djikstra:

http://youtube.com/watch?v=LmITN-wfn0w

```python

Dijkstra(G = (V, E), source)
    S = {} # S is the set of explored nodes
    pq = (0, source)
    while pq is not empty:
        if current_node in S:
            continue

        current_distance, current_node = pq.pop()
        d(current_node) = current_distance
        add current_node to S
        for each neighbour v of current_node:
            pq.push((current_distance + |(v, current_distance)|, v))

```

In [5]:
import numpy as np
import heapq

'''
Heapq (Priority Queue) import:

pq = []
heapq.heappush(pq, (1, "Praxis"))
heapq.heappush(pq, (1, "Calc"))
heapq.heappush(pq, (1, "C"))
print(heapq.heappop(pq))
'''

class Node:
    def __init__(self, value):
        self.value = value
        self.connections = []
        self.distance_from_start = np.inf

class Connection:
    def __init__(self, node: Node, weight: float):
        self.node = node
        self.weight = weight

def Dijkstra(start: Node, end: Node):
    start.distance_from_start = 0
    visited = set()
    to_be_considered = [(0, 
                        {"node": start, "prev": None})
                        ]
    current = start 
    
    while current != end:

        cur_dist, current_dict = heapq.heappop(to_be_considered)
        current = current_dict["node"]
        prev = current_dict["prev"]
        current.prev = prev 

        if current in visited:
            continue

        current.distance_from_start = cur_dist
        visited.add(current)

        for connection in current.connections:
            if connection.node in visited:
                continue
            heapq.heappush(to_be_considered, (connection.weight + current.distance_from_start, 
                        ({
                            "node": connection.node,
                            "prev": current
                        })))
    
    return current.distance_from_start

if __name__ == '__main__':
    yyz = Node('YYZ')
    lax = Node('LAX')
    yul = Node('YUL')
    yyy = Node('YYY')
    yul.connections.append(Connection(yyy, 100))
    yyz.connections.append(Connection(lax, 1000))
    yyz.connections.append(Connection(yul, 300))
    lax.connections.append(Connection(yul, 500))
    print(Dijkstra(yyz, yyy))
    
    # Backtrack to find the path
    cur = yyy
    while cur:
        print(cur.value)
        cur = cur.prev

400
YYY
YUL
YYZ


### Greedy Best-First Search


- `h(node)`: an estimate for how far the node is from the destination
    - A "heuristic function":

```python
GreedyBestFirst(G = (V, E), source, dest):
    S = {}
    v = source
    while v is not dest:
        select v from the neighbourhood of S with the smallest h(v)
        add v to S
```

- Not guaranteed to find the shortest path
- Will work well if `h(node)` is a good estimate

<img src="assets/gbfs.png"></img>

#### Djikstra vs. GBFS

<img src="assets/comparison.png"></img>

### A* Algorithm

Idea: Combine the ideas of Djikstra & GBFS
- Same as Djikstra but just one difference
    - When assigning a priority in pq.push(), add the distance between nodes plus the estimate `h(v)`

- Goal: find a cheap path from the start node to the end node
- Idea: explore a node that cheap to get to from start and looks like it's close to end
- Heuristic function: f(v) is small if v looks like it's close to end
- On a plane: can use the distance between the coordinates of v and end

```python
A*(G = (V, E), source, dest):
    S = dict()
    pq = (h(source), 0, source)
    
    while pq is not empty:
        if current_node in S:
            continue
        cur_node, cur_priority, cur_dist = pq.pop()
        dist(cur_node) = cur_dist
        add cur_node to S

        for each neighbour v of cur_node:
            dist = cur_dist + |(v, cur_dist)|
            pq.push(h(v) + dist, dist, v)
```
<br>
<img src="assets/a-star.png"></img>